# 制作TFrecord文件

In [ ]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob

In [ ]:
#获取image和json的id信息
image_list = sorted(os.listdir('img/pothole'))
json_list = sorted(os.listdir('json/pothole'))
print (len(image_list), len(json_list))

#判断是不是一一对应
image_list_new = []
json_list_new = []
for i in range(len(image_list)):
    a = image_list[i].strip('.jpg')
    b = json_list[i].strip('.json')
    image_list_new.append(a)
    json_list_new.append(b)
for i in range(len(image_list_new)):
    if image_list_new[i] != json_list_new[i]:
        print (image_list_new[i])

In [ ]:
'''
请注意替换下方相对路径，所有路径建议采用ipynb文件的相对路径
class_name.txt是全局类型定义文件，需要提前创建，保证与labelme解译的class_name里的病害类型名称一致
此处仅制作一个dataset,如果后续研究需要区分训练集、验证集、测试集，请分别制作。
'''
img_list = sorted(os.listdir('img/pothole'))

class_txt = open("class_name.txt","r")
class_name = class_txt.read().splitlines()

mask_path = 'mask/pothole'

for i in range(len(img_list)):
    path = "json_release/pothole/"+ img_list[i].split(".")[0] + "_json/label.png"
    #rint(path)
    mask_id =  img_list[i].split(".")[0]
    img = Image.open(path)
    img_1 = np.array(img)
    mask = np.zeros((1024,2048),dtype=int)
    with open("json_release/pothole/" + img_list[i].split(".")[0] + "_json/label_names.txt","r") as f:
        names = f.read().splitlines()
    for name in names:
        # index_json是x_json文件里存在的类label_names.txt，局部类
        index_json = names.index(name)
        # index_all是全局的类,
        index_all = class_name.index(name)
        mask[img_1==index_json] = index_all
    #lt.imshow(mask*30, cmap='gray', vmin=0, vmax=255)
    #lt.show()
    cv2.imwrite(os.path.join(mask_path , mask_id+'.png'), mask*30)
    #if i==10:
        #break

# 使用TFrecord数据集训练Unet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import tensorflow.keras as keras
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras import backend as K

import numpy as np

import PIL
from PIL import Image

import time

import os

import io
from tensorflow.python.data.experimental import AUTOTUNE

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
'''
加在TFrecord的必要函数
'''
def _parse_data(unparsed_example):
    return tf.io.parse_single_example(unparsed_example, image_feature_description)

def _bytestring_to_pixels(parsed_example):
    byte_string = parsed_example['image']
    image = tf.io.decode_image(byte_string)
    image = tf.reshape(image,[1024,2048])/255
    
    mask = tf.io.decode_raw(parsed_example['mask'], tf.uint8)
    mask = tf.reshape(mask,[1024,2048])/30#原来的灰度图是label*30的
    mask = tf.cast(mask,tf.int32)
    mask = tf.one_hot(mask, 7)#转one hot 编码
    
    return image, mask

def load_and_extract_images(filepath):
    dataset = tf.data.TFRecordDataset(filepath)
    dataset = dataset.map(_parse_data, num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(_bytestring_to_pixels, num_parallel_calls=AUTOTUNE)
    return dataset

In [ ]:
'''
载入数据
'''
image_feature_description = {
    "image": tf.io.FixedLenFeature([], tf.string), 
    "mask": tf.io.FixedLenFeature([], tf.string), 
    }

dataset_test = load_and_extract_images("pavementscape_test_2048_1024.tfrecords")
dataset_val = load_and_extract_images("pavementscape_val_2048_1024.tfrecords")
dataset_train = load_and_extract_images("pavementscape_train_2048_1024.tfrecords")

In [ ]:
'''
定义IoU损失函数
'''

def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)


def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)  # -1 ultiplied as we want to minimize this value as loss function

In [ ]:
'''
构建Unet结构
请依据自己数据集的图像尺寸情况调整IMG_WIDTH、IMG_HEIGHT、IMG_CHANNELS
请依据自己数据集中病害类型调整num_class
'''

IMG_WIDTH = 2048
IMG_HEIGHT = 1024
IMG_CHANNELS = 1
inputs_pixels = IMG_WIDTH * IMG_HEIGHT
num_class=6+1

inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

outputs = tf.keras.layers.Conv2D(num_class, (1, 1), activation='sigmoid')(c9)
#outputs = tf.argmax(outputs, axis=-1)


model_ns = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model_ns.compile(optimizer='adam', loss=[jacard_coef_loss], metrics=[jacard_coef])
#model_ns.compile(optimizer='adam', 
#             loss='CategoricalCrossentropy', #MeanAbsoluteError
#             metrics=['accuracy'])
model_ns.summary()

In [ ]:
filepath = ''#定义模型参数的保存路径
checkpoint_callback = ModelCheckpoint(
    filepath, monitor='val_accuracy', verbose=1,
    save_best_only=True, save_weights_only=True,
    save_frequency=1)#定义保存模型的节点


batch_size=4#Please choose your own batch size based on your GPU menory

model_ns.fit(dataset_train.batch(batch_size), epochs=300, verbose=1, shuffle=True, checkpoint_callback)#训练模型